<a href="https://colab.research.google.com/github/juanandres-montero/microeconometria/blob/main/Lab_007_Datos_De_Panel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <img src="https://github.com/juanandres-montero/microeconometria/blob/main/logo%20EEC%20grande.png?raw=1" width="396" height="161"><br>
    <b>EC4300 MICROECONOMETRÍA</b><br>
    <b>Asist: Juan Andrés Montero Zúñiga</b>
<br><br>
<b> Laboratorio #7:</b>
<br>    
<div style="font-size:250%;color:white; background-color: #0064b0;">Datos de Panel</div>
<div style="font-size:175%;color:white; background-color: #0064b0;">Efecto fijo, aleatorio y El test de Hausman</div>    

</center>
<br><br>
<p style="font-size:120%;">Este cuaderno toma como fuente la <i>Lecture 5</i> de la clase desarrolla por Phd. Randall Romero sobre Datos de panel. Se toma de
    <a href=""> Datos de Panel</a>
</p>
<i>
    Creado: 2024-May-11
</i>

<hr>

### 1. Descripción del problema

Queremos estimar el efecto de algunas variables (semanas trabajadas, educación y experiencia) sobre los salarios de 595 personas en Estados Unidos, utilizando datos del Panel Study of Income Dynamics (PSID) para el período de 1976 a 1982.

### 2. Preparativos

Se establecen algunos formatos y variables globales para facilitar el trabajo con los datos y la interpretación de los resultados:

use mus208psid , clear

set cformat %9.4f
set sformat %6.2f
set pformat %5.3f

global exogenas exp exp2 wks ed
global endogena lwage
global figure_size xsize(8) ysize(4)

In [ ]:
use mus208psid , clear

set cformat %9.4f
set sformat %6.2f
set pformat %5.3f

global exogenas exp exp2 wks ed
global endogena lwage
global figure_size xsize(8) ysize(4)


### 2.2 Resumen de los datos

Se realiza una limpieza y resumen de los datos:

- `keep $endogena $exogenas id t`: Conserva únicamente las variables de interés.
- `describe`: Proporciona una descripción estadística de las variables.
- `summarize`: Proporciona un resumen estadístico de las variables.
- `list id t $endogena $exogenas in 1/12, clean`: Lista las primeras 12 observaciones de las variables seleccionadas.
- `xtset id t`: Define la estructura de datos en panel, donde `id` es el identificador del individuo y `t` es el tiempo.
- `xtdescribe`: Describe la estructura de los datos en panel.
- `xtsum`: Proporciona un resumen estadístico de los datos en panel.

### 3. Estimación de modelos

Se estiman varios modelos de regresión para analizar el efecto de las variables explicativas sobre el salario.

#### 3.1 Regresión lineal

```stata
regress $endogena $exogenas, vce(cluster id) noci
estimates store OLS_robust
```

Se estima una regresión lineal simple y se almacena el resultado bajo `OLS_robust`.

#### 3.2 Efectos fijos

```stata
quietly xtreg $endogena $exogenas, fe noci
estimates store FE
xtreg $endogena $exogenas, fe vce(robust) noci
estimates store FE_robust
```

Se estima un modelo de efectos fijos y se almacenan los resultados, tanto con errores estándar robustos (`FE_robust`) como sin ellos (`FE`).

#### 3.2.1 Least-squares dummy variables

```stata
regress $endogena $exogenas i.id, vce(cluster id) noci
```

Se estima un modelo de efectos fijos utilizando variables dummy para cada individuo.

#### 3.3 Efectos aleatorios

```stata
quietly xtreg $endogena $exogenas, re noci
estimates store RE
xtreg $endogena $exogenas, re vce(robust) noci
estimates store RE_robust
```

Se estima un modelo de efectos aleatorios y se almacenan los resultados, tanto con errores estándar robustos (`RE_robust`) como sin ellos (`RE`).

### 3.4 Comparación de resultados

```stata
estimates table OLS_robust FE FE_robust RE RE_robust, b(%9.4f) se(%9.3f) stats(N r2 r2_o r2_b r2_w sigma_u sigma_e rho)
```

Se comparan los resultados de los distintos modelos en una tabla.

### 3.5 Test de Hausman

```stata
hausman FE RE, sigmamore
```

Se realiza el test de Hausman para comparar los modelos de efectos fijos y aleatorios y determinar cuál es más adecuado.
### 3.5.1 Interpretación del Test de Hausman

- **Hipótesis Nula (Ho)**: No hay diferencia sistemática entre los coeficientes de FE y RE (es decir, los estimadores de RE son consistentes).
- **Hipótesis Alternativa (Ha)**: Hay una diferencia sistemática entre los coeficientes de FE y RE (es decir, los estimadores de RE son inconsistentes).

### 3.5.2 Resultados del Test de Hausman

- **chi2(3) = 1513.02**: Este es el valor del estadístico de prueba.
- **Prob>chi2 = 0.0000**: El p-valor asociado con el estadístico de prueba.

### 3.5.3 Conclusión

Dado que el p-valor es 0.0000, que es menor que cualquier nivel de significancia típico (por ejemplo, 0.05, 0.01), rechazamos la hipótesis nula. Esto indica que existe una diferencia sistemática entre los coeficientes estimados por los modelos de efectos fijos y efectos aleatorios.

### 3.6 Modelo con efectos fijos de tiempo

```stata
xtreg $endogena $exogenas i.t, re
estimates store FE_t
estimates table FE FE_robust FE_t, b(%9.4f) se(%9.3f) keep($exogenas)
```

Se estima un modelo de efectos fijos incluyendo efectos fijos de tiempo y se comparan con los modelos previos.

Chi-cuadrado de Wald: 7166.37

Prob > chi2: 0.0000 (indicando que el modelo es globalmente significativo)

Corr(u_i, X): 0 (asumido)

